In [1]:
import blinkED as bk
import settings
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import scipy.signal as signal

In [7]:
df = pd.read_excel(r"C:\Users\cheta\Downloads\Charuni_TBUT_Trial1.xlsx")
use_data = df['CH1']
cutoff=0.05
b, a = signal.butter(2, cutoff, btype='lowpass') #low pass filter
filtered_data= signal.filtfilt(b, a, use_data)
eog_cleaned = bk.eog_clean(filtered_data, sampling_rate=100, method='neurokit')

blinks = bk.signal_findpeaks(eog_cleaned, relative_height_min=2.5)
print(f"{len(blinks['Peaks'])} Blinks found")
df_blinks = pd.DataFrame(blinks)
df_blinks.head()

# Plot segmented blinks
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(eog_cleaned)), y=eog_cleaned, name='EOG signal'))

for i, peak in enumerate(blinks['Peaks']):
    start = peak    # Subtract 50 samples from start to show some lead-up to blink
    end = peak  # Add 50 samples to end to show some post-blink activity
    
    blink_segment = eog_cleaned[start:end]
    blink_times = np.arange(start, end)
    
    fig.add_trace(go.Scatter(x=blink_times, y=blink_segment, name='Blink' ))
    fig.add_trace(go.Scatter(x=[peak], y=[eog_cleaned[peak]], mode='markers', name='Blink peak' ))

fig.update_layout(xaxis_title='Sample', yaxis_title='EOG amplitude', title='Segmented Peaks')
fig.update_xaxes(rangeslider_visible=True)
fig.show()


5 Blinks found


In [8]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go

# Clean EOG data
eog_cleaned = bk.eog_clean(filtered_data, sampling_rate=100, method='neurokit')

# Find blinks
blinks = bk.signal_findpeaks(eog_cleaned, relative_height_min=2.5)
print(f"{len(blinks['Peaks'])} Blinks found")
df_blinks = pd.DataFrame(blinks)
df_blinks.head()

# Plot segmented blinks
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(eog_cleaned)), y=eog_cleaned, name='EOG signal'))

# Iterate through blinks and plot them
for i, peak in enumerate(blinks['Peaks']):
    start = peak
    end = peak
    
    # Find start and end of blink
    while start > 0 and eog_cleaned[start-1] < eog_cleaned[start]:
        start -= 1
    while end < len(eog_cleaned)-1 and eog_cleaned[end+1] < eog_cleaned[end]:
        end += 1
    
    # Determine if blink is incomplete
    peak_value = eog_cleaned[peak]
    if peak_value < 10 * (eog_cleaned[start] + eog_cleaned[end]):
        color = 'red'  # Incomplete blink
    else:
        color = 'red'  # Complete blink
    
    # Plot blink segment and blink peak
    blink_segment = eog_cleaned[start:end]
    blink_times = np.arange(start, end)
    fig.add_trace(go.Scatter(x=blink_times, y=blink_segment, name='Blink', line=dict(color=color)))
    fig.add_trace(go.Scatter(x=[peak], y=[peak_value], mode='markers', name='Blink peak', marker=dict(color=color)))

fig.update_layout(xaxis_title='Sample', yaxis_title='EOG amplitude', title='Segmented blinks')
fig.update_xaxes(rangeslider_visible=True)
fig.show()
peaks = bk.eog_findpeaks(eog_cleaned, sampling_rate=250)
print(len(peaks))
info = bk.eog_features(eog_cleaned, peaks, sampling_rate=250)
df = pd.DataFrame(info)
df.head()

5 Blinks found


8


,Blink_LeftZeros,Blink_RightZeros,Blink_pAVR,Blink_nAVR,Blink_BAR,Blink_Duration,Interblink Duration
0,124,180,0.036974,-0.002537,23.659192,0.224,5.040
1,1392,1446,0.012480,-0.000339,2.432586,0.216,8.636
2,3544,3599,0.028318,-0.000362,7.936845,0.220,43.056
3,14309,14367,0.011218,-0.000333,2.603981,0.232,1.020
4,14551,14622,0.005149,-0.000294,6.232894,0.284,11.968


4


,Blink_LeftZeros,Blink_RightZeros,Blink_pAVR,Blink_nAVR,Blink_BAR,Blink_Duration,Interblink Duration
0,657,760,0.004758,0.000386,15.426301,0.412,17.616
1,5075,5137,0.036913,-0.001095,130.909835,0.248,21.628
2,10482,10548,0.033351,-0.000278,245.346098,0.264,21.004
3,15727,15795,0.028058,-0.000547,66.882564,0.272,NaN
